<a href="https://colab.research.google.com/github/mrchapagain/SocialNetworkAnalytics/blob/main/TwiteerDataAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tweeter Data Analytics
***Twiteer APIs:***
*   *Search API*
*   *Ads API*
*   *Streaming API*

***Class object for handling data***

In [ ]:
from tweepy.streaming import StreamListener
import json
import time
import sys

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')

***Libaries***

In [ ]:
# Necessary libaries
from tweepy import OAuthHandler
from tweepy import API

***Necessary information form tweet account***
*   access_token
*   access_token_secret
*   consumer_key
*   consumer_secret

***Tweepy authentication***

In [ ]:


# Consumer key authentication
auth = OAuthHandler(consumer_key, consumer_secret)

# Access key authentication
auth.set_access_token(access_token, access_token_secret)

# Set up the API with the authentication handler
api = API(auth)
api

***Collecting data with tweepy***

In [ ]:
from tweepy import Stream

# Set up words to track
keywords_to_track = ["#rstats", "#python"]

# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begein collecting data
tweet_streem= stream.filter(track = keywords_to_track)
type(stream)

Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403
Encountered error with status code: 403


***Contents of Twitter JSON***
*   How many retweets,favorites
*   Language
*   Reply to which tweet
*   Reply to which user

***Loading and accessing tweets***

In [ ]:
# Load JSON
import json
tweet_json = open(tweet_streem, 'r').read()
# Convert from JSON to Python object
tweet = json.loads(tweet_json)

# Print tweet text
print(tweet["text"])

# Print tweet id
print(tweet["id"])
tweet_json = open('tweet-example.json', 'r').read()


TypeError: ignored